# Preparation for class in week 5


Our example is inspired by the Dutch LISS data, in particular the waves on Time Use and consumption in November 2019 and in April 2020. 

In particular, the data contain the following variables, alphabetically sorted:

| Variable     | Content                                                           |
|:-------------|:------------------------------------------------------------      |
| geslacht     | Gender (Man: Male, Vrouw: Female)                                 |
| nohouse_encr | Household identifier                                              |
| nomem_encr   | Member identifier                                                 |
| v1q1_v1col1  | Working hours (Nov) / Working hours at workplace (Apr)            |
| v1q1a_v1col1 | Working hours in home office, no kids in HH (Apr)                 |
| v1q1b_v1col1 | Working hours in home office while responsible for kids (Apr)     |
| v1q1c_v1col1 | Working hours in home office while not responsible for kids (Apr) |
| v1q5_v1col1  | Childcare hours (all in Nov, residual in Apr)                     |
| v1q5a_v1col1 | Homeschooling hours (Apr)                                         |


In this exercise, we will only work with the November data.

We start by importing Pandas

In [ ]:
import pandas as pd

## Read in the data

The November data is stored in Stata format. Replace the `XXXXX` in the next cell by the appropriate Pandas function.

In [ ]:
nov_2019 = pd.read_stata("time_use_2019-11.dta", convert_categoricals=True)

## Browse the data

You can browse through the data by just typing the name of a DataFrame as the last thing in a cell.

This will yield a nice html-formatted output. Use this to find out the difference between setting `convert_categoricals` to `True` or `False` in the call above. In case you know some Stata: Can you explain what is happening? Else don't worry about the reasons behind what is going on.


In [ ]:
nov_2019

## Rename variables

We give the Dutch and partly cryptic variable names sensible identifiers. The `replace` method on DataFrames returns a new DataFrame, so we need to assign it to `nov_2019` again if we want to continue working with it.

Note that this is a very stateful transformation if you assign to the same variable; you will not be able to successfully execute the cell twice without re-loading the data above.

In [ ]:
nov_2019 = nov_2019.rename(
    columns={
        "geslacht": "gender",
        "v1q1_v1col1": "working_hours",
        "v1q5_v1col1": "childcare_hours",
    }
)

## Convert data types

`nomem_encr` and `nohouse_encr` are classical identifiers. They can be used to identify a particular observation, but they do not carry any meaning beyond that.

You have heard in the screencast that you should never use floating point numbers for identifiers. It is common that this happens in Stata, though (e.g., through use of `compress` or mathematical operations of integers, which do implicit type conversions unless you request an integer back).

Add new columns `hh_id` and `ind_id` with sensible data types. 


In [ ]:
nov_2019["hh_id"] = ...

## Select columns

We do not need to keep the old identifiers anymore.

You can select a subset of columns by including a list of column names in the standard square brackets used for indexing in Python.

Replace the XXXX and YYYY appropriately and include all other variables that you want to keep.

In [ ]:
nov_2019 = nov_2019[["XXXX", "YYYY"]]

## Set index

The default index created by Pandas (a DataFrame always has an index) does not make much sense.

Create an index based on a column / several columns that makes sense to you via replacing the XXXX by an appropriate construct.

In [ ]:
nov_2019_with_index = nov_2019.set_index(XXXX)

## Our first reduction operation

Calculate the mean hours spent on different activities.

## Our second reduction operation

Calculate the mean hours spent on different activities by gender by appending the following with the appropriate method to calculate a mean.

In [ ]:
nov_2019.groupby("gender")[["working_hours", "childcare_hours"]]